In [ ]:
#!git clone https://github.com/fchollet/keras.git && cd keras && python setup.py install --user

In [ ]:
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K 
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
import numpy as np
import os

In [ ]:

images = []

def load_data
for filename in os.listdir('/data/train_set'):
    if filename[-3:] == "npz":
        image = np.load('/data/train_set'+filename)
        images.append(image['features'])
images = np.array(images, dtype=float)
images = preprocess_input(images)

IR2 = InceptionResNetV2(weights='imagenet', include_top=False)
features = IR2.predict(images)
print(features.shape)

In [ ]:
num_words = 19
tokenizer = Tokenizer(num_words=num_words, filters='', split="\n", lower=False)

def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

X = []
for filename in os.listdir('/data/train_set'):
    if filename[-3:] == "gui":
        X.append('<START>' + load_doc('/data/train_set'+filename) + '<END>')
    
tokenizer.fit_on_texts([load_doc('syntax.vocab')])

vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(X)
max_length = max(len(s) for s in sequences)
 
X, y, image_data = list(), list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        image_data.append(features[img_no])
        X.append(in_seq)
        y.append(out_seq)

X, y, image_data = np.array(X), np.array(y), np.array(image_data)

In [ ]:
image_features = Input(shape=(6, 6, 1536,))
image_flat = Flatten()(image_features)
image_flat = Dense(128, activation='relu')(image_flat)
ir2_out = RepeatVector(max_length)(image_flat)

language_input = Input(shape=(max_length,))
language_model = Embedding(vocab_size, 32, input_length=max_length, mask_zero=True)(language_input)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = TimeDistributed(Dense(128, activation='relu'))(language_model)

decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[image_features, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
model.fit([image_data, X], y, batch_size=1, shuffle=False, epochs=5)